# Course 2: Sensor Fusion
## Part 4: Unscented Kalman Filter (UKF)
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

## 1. Introduction

### 1.1. Unscented Kalman Filter (UKF)

#### Background

In this lesson, we will be building on our knowledge of the [extended Kalman filter](https://en.wikipedia.org/wiki/Extended_Kalman_filter) (EKF) in order to implement an [Unscented Kalman filter](https://en.wikipedia.org/wiki/Kalman_filter#Unscented_Kalman_filter)
 (UKF) for object tracking with non-linear process and measurement models.
 
Rather than linearising a non-linear process model, the Unscented Kalman filter use a deterministic sampling technique known as an [unscented transformation](https://en.wikipedia.org/wiki/Unscented_transform) to pick a minimum set of sample points (known as sigma points) around the mean of the covariance. With these point estimates we evaluate the original non-linear process model $\mathcal{f}$ to estimate a new mean and covariance. The UKF approach is often more accurate than the extended Kalman filter (EKF) and avoids the computation of the Jacobians.

#### From Extended- to Unscented Kalman Filter

The UKF uses the same prediction and update equations found in the extended Kalman filter. The difference between the two, however, is how the UKF deals with non-linear process and measurement models. Rather than linearising the non-linear functions using a first-order Taylor expansion and deriving a Jacobian matrix, we instead sample a set of "sigma points" from a Gaussian distribution centred about the mean of the covariance given for the non-linear process model.

#### UKF Process Chain

Given a state vector mean $x_{k\vert k}$ and state covariance $\mathrm{P}_{k\vert k}$ at time-step $k$. Our goal with the UKF is to predict the mean state $x_{k+1 \vert k}$ and covariance $\mathrm{P}_{k+1\vert k}$ to the next time-step $k+1$. Note that here we use the pipe "$\vert$" notation to represent on the left-hand side the current estimated time-step, and on the right-hand side the latest considered measurement. For $x_{k\vert k}$ and $\mathrm{P}_{k\vert k}$, we say that the estimation is for time-step $k$ and the measurement at time-step $k$ has already been taken into account — this is commonly referred to as the _posterior estimation_. On the other hand, for $x_{k+1\vert k}$ and $\mathrm{P}_{k+1\vert k}$, we have the estimation for time-step $k+1$ which takes into account the measurement from the previous time-step $k$ — this is commonly referred to as the _a priori estimation_.

<img src="figures/Exercises/2023-01-13-Figure-1-Unscented-Kalman-Filter-Estimation.png" alt="Figure 1. The Unscented Kalman Filter — Posterior and A priori distributions formed by the state mean and covariance before- and after the estimation at time-step k+1.">

$$
\begin{align}
\textrm{Figure 1. The Unscented Kalman Filter — Posterior and A priori distributions formed by the state mean and covariance before- and after estimation at time-step k+1.}
\end{align}
$$

The two ellipses in Figure 1 above visualise the distribution of the uncertainty; all points along the perimeter of the ellipse have the same probability density. If the uncertainty is normally-distributed these points form the shape of the ellipse, as shown above (often called the "error ellipse"). The _A priori_ estimation is what is obtained after the prediction step of the UKF has been performed. These two ellipses can be seen as the visualisation of the covariance matrix $\mathrm{P}$. Assuming a linear process model, we have a prediction step given by the equations:

$$
\begin{align}
x_{k+1} &= \mathrm{F}x_{k} + \nu_{k}, \\
\mathcal{Q} &= E\left[\nu_{k} \cdot \nu_{k}^{\top}\right].
\end{align}
$$

Here, $\mathcal{Q}$ is the covariance matrix of the process noise. In the linear case, the Kalman filter is used to solve the prediction problem. A solution can be found with the following set of equations:


$$
\begin{align}
x_{k+1 \vert k} &= \mathrm{F}x_{k \vert k}, \\
\mathrm{P}_{k+1 \vert k} &= \mathrm{F}\mathrm{P}_{k\vert k}\mathrm{F}^{\top} + \mathcal{Q}.
\end{align}
$$

When the process model is non-linear, as in the case of our problem, we have the prediction step given as:

$$
\begin{align}
x_{k+1} &= \mathcal{f}\left(x_{k} + \nu_{k}\right), \\
\mathcal{Q} &= E\left[\nu_{k} \cdot \nu_{k}^{\top}\right].
\end{align}
$$

Since the prediction step of a non-linear process model results in a non-linear distribution, it is no longer possible to directly represent the predicted state distribution, i.e., the a priori estimation, as a normal distribution forming an ellipse. We no longer have an analytical path, and therefore we need to find an approximate normal distribution that represents the original non-linear distribution as closely as possible. In order to do so, we find a normal distribution characterised by the same mean value and the covariance matrix as the _real_ predicted distribution. The question is, how do we derive this mean vector and this covariance matrix? This is what the unscented transformation does for us.

###### Unscented Transformation

To deal with non-linear functions, the UKF uses an [unscented transformation](https://en.wikipedia.org/wiki/Unscented_transform). The unscented transformation provides the mean state vector and covariance matrix needed to estimate the a priori normal distribution. By using sigma points, we are able to approximate the non-linear distribution using only a set of point estimates. These point estimates, called sigma points, are transformed into the predicted state-space using the non-linear process model function $\mathcal{f}$. The sigma points are chosen relative to the mean state and are defined with respect to the standard deviation signal of every state dimension. This is why these points are called _sigma_ points; they serve as representatives of the whole distribution.

Once the set of sigma points has been transformed into the predicted state-space by the non-linear function $\mathcal{f}$, all that is left to do is calculate the mean and covariance of the sampled sigma points. The mean and covariance of the sigma points serves as only an approximation of the _real_ predicted distribution — but in most cases we get a useful estimation to work with.

Another great thing about the unscented transformation and sigma point estimation is that, in the linear case, the _exact_ solution to the real predicted distribution can be derived. Thus, for a linear process model, the sigma points provide the exactly the same solution as the standard Kalman filter. Pretty neat, huh? While all this sounds great, using sigma points in the linear case isn't an entirely sound decision, since their added complexity weighs things down in terms of calculation time.

As a recap, sigma points provide a way to estimate a non-linear distribution, providing the best possible mean and covariance estimates for a normal distribution assumption. Now, with the general idea of sigma points out of the way, let's dive into the meat of the Unscented Kalman filter — the _prediction_ and _update_ steps.

###### Prediction and Innovation of Sigma Points

Starting with a state mean vector $x_{k \vert k}$ and covariance $\mathrm{P}_{k \vert k}$, the prediction and update (also called _innovation_) steps of the UKF give us the updated state vector $x_{k+1 \vert k}$ and covariance matrix $\mathrm{P}_{k+1 \vert k}$ predicted into the next time-step $k+1$. This is essential to applications such as object tracking where we must reliably estimate the change in vehicle position and orientation over a given time period $\Delta t$.

Since we begin the UKF process chain with the _prediction_ step, we split this into three parts: first, we need to know a good way to choose the sigma points. Secondly, we need to know how to predict the sigma points into the actual state-space of the process model. Lastly, we need to know how to calculate the prediction mean and covariance from the sigma point estimations.

We start with prediction off with the posterior state $x_{k\vert k}$ and the posterior covariance matrix $\mathrm{P}_{k \vert k}$ of the vehicle from the previous iteration. They represent the distribution of our vehicle's current state. This distribution is a linear, normal distribution from which we can generate our sigma points. The number of sigma points we generate at this step depends on the dimension of the state vector $n_{x}$. Since we describe in Sect 1.2. the CTRV model with state vector of dimension $5$, we say here that $n_{x} = 5$. We choose a number of sigma points

$$
\begin{align}
n_{\sigma} &= 2*n_{x} + 1 \\
\end{align}
$$

which, for the CTRV model gives us a total of $n_{\sigma} = 2*\left(5\right) + 1 = 11$ points to generate. This corresponds to _one_ sigma point to represent the mean of the state, and another two sigma points for each dimension of the state. Each of these two sigma points are spread in opposite directions, and their "spread" is determined by a design parameter $\lambda$ which governs their distance from the centre of the distribution (the mean point). Assuming a state vector of only the position $x_{k} = \left[p_{x} p_{y}\right]$, we can illustrate the rule for the sigma point matrix as:

$$
\begin{align}
\mathcal{X}_{k\vert k} &= \begin{bmatrix} x_{k\vert k} & x_{k\vert k} + \sqrt{\left(\lambda + n_{x}\right)\mathrm{P}_{k\vert k}} & x_{k \vert k} - \sqrt{\left(\lambda + n_{x}\right)\mathrm{P}_{k\vert k}}\end{bmatrix},
\end{align}
$$

such that the columns of the sigma point matrix $\mathcal{X}_{k\vert k}$ contain the set of $n_{\sigma} = 5$ sigma points corresponding to the state dimension of the position coordinates of $n_{x} = 2$. The "spreading factor" in the square-root $\left(\lambda + n_{x}\right)$ governs the spread of the sigma points, where $\lambda = 3 - n_{x}$ is commonly chosen to compute this value. While it isn't directly relevant here, we will be using the [Cholesky decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition#Kalman_filters) to compute the square-root of the covariance matrix $\mathrm{P}_{k \vert k}$ as given by the decomposition $\mathrm{A} = \mathrm{L}\mathrm{L}^{\top}$ where the columns of $\mathrm{L}$ are added and subtracted to the mean $x$ to form a set of $2*n_{x}$ vectors. Here, $\mathrm{L}$ is the [lower triangular matrix](https://en.wikipedia.org/wiki/Lower_triangular_matrix) and, when $\mathrm{A}$ is assumed to be a real (hence, symmetric positive-definite) matrix, $\mathrm{L}$ is has real-valued positive entries along the diagonal. Also important to note for our later implementation of this sigma point generator function is that the first column of the matrix $\mathcal{X}_{k\vert k}$ is the mean of the distribution $x_{k\vert k}$, the second through $n_{x} + 1$ column will be given by the second term above (with the addition sign between the mean and the square-root term), and the last $n_{x} + 2$ through $2 * n_{x} + 1$ columns will be given by the third term above (with the subtraction sign between the mean and the square-root term).

##### UKF Augmentation

Now that we know how to represent the uncertainty of the posterior state estimation using sigma points, accomplishing the second step of the UKF _prediction_ is nearly complete. Just plug in each sigma point into the non-linear process function $\mathcal{f}$ to obtain the next-state estimation:

$$
\begin{align}
x_{k+1} &= \mathcal{f}\left(x_{k}, \nu_{k}\right).
\end{align}
$$

However, we have yet to consider the effect of the non-linear process noise vector $\nu_{k}$. Fortunately, with the UKF we are able to handle non-linear process noise in a step known as _UKF augmentation_. To start, we recall the expression of the independent noise processes as stochastic properties (this will be covered in the next section for the CTRV model). From here, we write the process noise covariance matrix as the expression:

$$
\begin{align}
\mathcal{Q} &= E\left\{\nu_{k}\cdot \nu_{k}^{\top}\right\} = \begin{bmatrix}\sigma_{a}^{2} & 0 \\ 0 & \sigma_{\ddot{\psi}^{2}}\end{bmatrix},
\end{align}
$$

which is a matrix that contains the variances of the noise processes for the longitudinal acceleration and change in yaw angle. Here, we consider the white-noise processes independent from each other (hence, the $0$ values along the diagonal).

In the augmentation step, we form the _augmented_ state $x_{a}$ by adding the noise vector $\nu_{k}$ to the state vector $x_{k}$. Repeating the sigma point generation for this augmented state vector $x_{a, k}$ and covariance matrix $\mathrm{P}_{a, k \vert k}$, we obtain a new set of sigma points representing the uncertainty caused by the sigma points. Note that the number of augmented sigma points computed here differs than in the first calculation, since we have an additional two terms added to the augmented state vector, resulting in the dimensions $n_{a} = n_{x} + 2$. From our earlier example, we obtain $n_{\sigma_{a}} = 2*7 + 1 = 15$ augmented sigma points. This results in an augmented covariance matrix $\mathrm{P}_{a, k\vert k}$ of dimensions $7 \times 7$ such that

$$
\begin{align}
\mathrm{P}_{a, k\vert k} &= \begin{bmatrix} \mathrm{P}_{k, \vert k} & 0 \\ 0 & \mathcal{Q} \end{bmatrix},
\end{align}
$$

where the upper-left of the matrix is given as the original covariance matrix, and the lower-right of the matrix is the independent process noise covariance matrix defined above. The remaining values in the matrix are therefore set to zero.

Lastly, we compute these sigma points in the exact same way as before. Just evaluate the expression for the previous sigma point matrix to get the augmented sigma point matrix $\mathcal{X}_{a, k\vert k}$. No need to modify the scaling factor on this one. Once the augmented sigma point matrix has been formed, simply plug these points into the process model $\mathrm{f}$ using the process noise vector $\nu_{k}$ to obtain the predicted sigma point matrix $\mathcal{X}_{k+1\vert k}$. You'll see in the following Sect. 1.2 the derivation of the state-space form of the process model $\mathcal{f}$ and the expression for the next-state prediction, which we will evaluate in order to obtain this matrix.

##### Prediction and Innovation of Mean and Covariance

Before jumping into the innovation step of the UKF, we finish off the prediction step by calculating the mean and covariance of the predicted state from the augmented sigma points we just derived. The standard rule for calculating the mean and covariance of a group of state samples is given by the following equations:

$$
\begin{align}
x_{k+1\vert k} &= \sum_{i=0}^{n_{a}} w_{i}\mathcal{X}_{k+1\vert k, i}, \\
\mathrm{P}_{k+1 \vert k} &= \sum_{i=0}^{n_{a}} w_{i} \left(\mathcal{X}_{k+1\vert k, i} - x_{k+1\vert k}\right)\left(\mathcal{X}_{k+1\vert k, i} - x_{k+1\vert k}\right)^{\top}.
\end{align}
$$

Here, we compute the sum over the columns of the predicted sigma point matrix (hence, the sum from $i=0$ to $i=n_{a}$ where $n_{a}$ corresponds to the number of augmented sigma points). In these expressions is the use of a weight vector $w$ with values given by:

$$
\begin{align}
w_{i} &= 
\begin{cases}
    \frac{\lambda}{\lambda + n_{\mathrm{aug}}}, & \text{if} \quad i = 0 \\
    \frac{1}{2}\left(\lambda + n_{\mathrm{aug}}\right), & \text{otherwise}. \\
\end{cases}
\end{align}
$$

where the weight expression differs for the first element of the vector and all other elements $i = 1,\ldots, n_{\mathrm{aug}}$. Note that the dimension $n_{\mathrm{aug}}$ is the dimension of the augmented state vector, which previously we assumed to be $n_{\mathrm{aug}} = n_{x} + 2$. These weight values also depend on the spreading parameter $\lambda$ which is used in the sigma point generation. This is to recover the covariance by inverting the spreading of the sigma points. There's more than one way to define these weights in literature, but for our implementation we use the expressions above.

##### Prediction of the Measurement

We have successfully arrived at the first step of the UKF _innovation_ / update process. Here, our goal is to transform the predicted state into the measurement space. The function that defines this transformation is the _measurement model_. Of course, here is now where we must consider the type of measurement we are transforming the predictions with respect to. For our later implementation we will be using a radar sensor model whose measurement vector $z_{k+1\vert k}$ and expression of the measurement function $\mathcal{h}$ are given as:

$$
\begin{align}
z_{k+1\vert k} &= \begin{bmatrix} \rho & \phi & \dot{\rho} \end{bmatrix}^{\top}, \\
\
z_{k+1} &= \mathcal{h}\left(x_{k+1}\right) + \omega_{k+1}.
\end{align}
$$

Here, the $\rho$ term is the measured radial distance ($\mathrm{m}$), the $\phi$ term is the heading angle ($\mathrm{rad}$), and $\dot{\rho}$ as the rate-of-change of radial distance (i.e., velocity, $\mathrm{m}/\mathrm{s}^{2}$).

This transformation process is very similar to the problem we had in the prediction step; we need to transform a distribution through a non-linear function $\mathcal{h}$. Therefore, we apply the exact same unscented transformation approach as we did before during the state prediction. However, this time we take two shortcuts here: (1) we repeat the sigma point generation using the predicted mean and covariance matrix, and (2) hold out computing the effect of measurement noise on the radar model.

Along the path of shortcut (1) we re-use the sigma points we have already generated from the prediction step and skip generating new sigma points all-together for the radar measurement. This allows us to also skip the augmentation step entirely with shortcut (2), since we assume here a purely _linear_ additive effect of measurement noise on the radar measurement model. This gives an easier way to consider measurement noise which we will jump back to later in the calculation of the measurement covariance matrix using matrix $\mathrm{R}$. So for now, set the $\omega_{k+1}$ term to be $0$.

With these two simplifications, we only have to transform the individual sigma points we previously derived into the radar measurement space. Then, we use them to calculate the mean $z_{k+1\vert k}$ and covariance $\mathrm{S}$ of the predicted radar measurement. The re-used sigma points, after using them to evaluate the measurement model $\mathcal{h}$, are stored in the radar sigma point matrix $\mathcal{Z}_{k+1\vert k}$.

Applying the equations to our original problem formulation, we obtain a predicted sigma point matrix $\mathcal{X}_{k+1 \vert k}$ of dimensions $5 \times 15$ and a transformed sigma point matrix into measurement space $\mathcal{Z}_{k+1\vert k}$ of dimensions $3 \times 15$. The last step we have to do involving these sigma point matrices is to compute the predicted measurement mean $z_{k+1 \vert k}$ and the predicted measurement covariance $\mathrm{S}_{k+1\vert k}$, as follows:

$$
\begin{align}
z_{k+1\vert k} &= \sum_{i=1}^{n_{\sigma}} w_{i}\mathcal{Z}_{k+1\vert k, i}, \\
\mathrm{S}_{k+1\vert k} &= \sum_{i=0}^{n_{\sigma}} w_{i}\left(\mathcal{Z}_{k+1 \vert k, i} - z_{k+1\vert k}\right)\left(\mathcal{Z}_{k+1\vert k, i} - z_{k+1\vert k}\right)^{\top} + \mathrm{R}. \\
\end{align}
$$

In the above prediction equations we have the measurement noise covariance $\mathrm{R}$ which follows $\mathrm{R} = E\{\omega_{k} \cdot \omega_{k}^{\top}\}$. This applies to our case since the measurement noise of the radar sensor is linear and has a purely additive effect on the measurement model. Therefore, we obtain the covariance as the following:

$$
\begin{align}
\mathrm{R} &= E\{\omega_{k} \cdot \omega_{k}^{\top}\} = \begin{bmatrix}\sigma_{\rho}^{2} & 0 & 0 \\ 0 & \sigma_{\phi}^{2} & 0 \\ 0 & 0 & \sigma_{\dot{\rho}}^{2}\end{bmatrix}.
\end{align}
$$

##### UKF Update Step

TODO.

##### Summary

TODO.

### 1.2. Constant Turn Rate and Velocity Magnitude (CTRV) Model

#### Background

As with most constant velocity models, the Constant Turn Rate and Velocity Magnitude (CTRV) model is a simplification of how objects actually navigate in real-world environments. The CTRV model assumes that an object (here, a vehicle), moves along a path with a constant turn-rate and constant velocity. Rather than a strict constant velocity, constant direction model, we allow the vehicle to travel a curved path with a constant heading / yaw angle. This additional assumption allows us to represent vehicle motion on roads with turns.  

We will see in Figure 1 in the later part of this section that here we approximate the vehicle's constant turn-rate $\psi$ with respect to a constant velocity $v$. In Figure 1 this is shown as a line tangential to the vehicle's actual (assumed) motion along a curved path. For now, just know that the CTRV model provides a decent starting point for representing vehicle behaviour in real traffic scenarios.

#### State equations

Using the CTRV model, a vehicle state at a given point in time can be represented with the following state vector: 

$$
\begin{align}
x = \begin{bmatrix}p_{x} & p_{y} & v & \psi & \dot{\psi}\end{bmatrix}^{\top},
\end{align}
$$

where the 2D vehicle position is given by $\left(p_{x}, p_{y}\right)$, the magnitude of velocity $v$, the orientation $\psi$ (i.e., the yaw angle of the velocity component), and the rate-of-change of the orientation $\dot{\psi}$.

##### Process model
In order to represent a state change of the vehicle from time-step $k$ to $k+1$, we use the transition function:
$$
\begin{align}
x_{k+1\vert k} &= \mathcal{f}\left(x_{k}, \nu_{k}\right).
\end{align}
$$

The goal of the process model function represented above is to predict the state of the vehicle $x_{k+1 \vert k}$ at the new time-step. To evaluate the process model $\mathcal{f}$, we use the current vehicle state $x_{k}$ and the process noise $\nu_{k}$ as inputs. In order to compute the state change, we must form an expression for the change-rate of the state $\dot{x}$, given as:
$$
\begin{align}
\dot{x} &= \begin{bmatrix}\dot{p}_{x} & \dot{p}_{y} & \dot{v} & \dot{\psi}, & \ddot{\psi} \end{bmatrix}^{\top}.
\end{align}
$$

From this relation we can directly derive the change-rate $\dot{x}$ in terms of a differential equation $\dot{x} = g\left(x\right)$.

We can re-write each of the terms of the state differential independently of any of the state elements in $x$. For example, we can write $\dot{p}_{x}$, i.e., the change in velocity along the $x$-axis, as:
$$
\begin{align}
\dot{p}_{x} = v_{x} = \cos(\psi)\cdot v,
\end{align}
$$

assuming a constant direction from the current vehicle state $x_{k}$ and the predicted vehicle state $x_{k+1}$. With this, we form a trigonometric relation an derive the expressions of the other state variables as follows:


$$
\begin{align}
\dot{x} &= 
\begin{bmatrix} \dot{p}_{x} \\ \dot{p}_{y} \\ \dot{v} \\ \dot{\psi} \\ \ddot{\psi} 
\end{bmatrix} = 
\begin{bmatrix}
\cos(\psi) \cdot v \\ \sin(\psi) \cdot v \\ 0 \\ \dot{\psi} \\ 0 
\end{bmatrix}.
\end{align}
$$

Since we assume a constant turn-rate and a constant velocity, our expression for the rate-of-change of both these state variables is therefore equal to $0$.

In order to form an expression for the state change over time, we must define our time-step $\Delta t$ as the change in time from $t_{k}$ to $t_{k+1}$. Our expression for the next-state $x_{k+1}$ is therefore:
$$
\begin{align}
x_{k+1} &= x_{k} + \int_{t_{k}}^{t_{k+1}} \begin{bmatrix} \dot{p}_{x}(t), \  \dot{p}_{y}(t), \ \dot{v}(t), \ \dot{\psi}(t), \ \ddot{\psi}(t) \end{bmatrix}^{\top}dt.
\end{align}
$$

Expanding the integral and replacing the derivative terms in the next-state vector, then solving for the integral terms, we obtain:

$$
\begin{align}
x_{k+1} &= x_{k} + \begin{bmatrix}v_{k}\int_{t_{k}}^{t_{k+1}} \mathrm{cos}\left(\psi_{k} + \dot{\psi} \cdot\left(t - t_{k}\right)\right) dt 
\\ v_{k}\int_{t_{k}}^{t_{k+1}} \mathrm{sin}\left(\psi_{k} + \dot{\psi} \cdot\left(t - t_{k}\right)\right) dt 
\\ 0 
\\ \dot{\psi}_{k}\cdot \Delta t 
\\ 0 \end{bmatrix}
= 
\begin{bmatrix}
\frac{v_{k}}{\dot{\psi}_{k}}\left(\mathrm{sin}\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) - \mathrm{sin}\left(\psi_{k}\right)\right)
\\ \frac{v_{k}}{\dot{\psi}_{k}}\left(-\mathrm{cos}\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) + \mathrm{cos}\left(\psi_{k}\right)\right)
\\ 0
\\ \dot{\psi}_{k} \cdot \Delta t
\\ 0
\end{bmatrix}.
\end{align}
$$

When the rate-of-change of the yaw angle $\dot{\psi}_{k} = 0 \ \mathrm{rad}/\mathrm{s}^{2}$, the first two terms of the final state vector are reduced to $v_{k}\cos\left(\psi_{k}\right)\cdot \Delta t$ and $v_{k}\tan\left(\psi_{k}\right)\cdot \Delta t$, respectively. 

<img src="figures/Exercises/2023-01-13-Figure-2-Constant-Turn-Rate-Constant-Velocity-Model.png" alt="Figure 2. The Constant Turn Rate Constant Velocity (CTRV) model visualised.">

$$
\begin{align}
\textrm{Figure 2. The Constant Turn Rate and Velocity Magnitude model.}
\end{align}
$$

In the above figure we depict the CTRV model assumption applied to the vehicle state at time-step $k$ to time-step $k+1$. The state transition is given by the process model $f$ which predicts the next-state of the vehicle at time $k+1$ using a constant velocity assumption — that is, assuming the vehicle proceeds to the next time-step with a constant heading angle $\psi$ and a constant velocity $v$. The CTRV model assumption allows us to simplify the calculation of the velocity to its magnitude rather than in either the $x$- and $y$-direction. The CTRV model gives us a basis for the estimation by exploiting the geometry of the assumed next-state trajectory. With a rate-of-change of the yaw angle $\ddot{\psi}$, our state transition equation with vector $\dot{x}$ becomes:

$$
\begin{align}
x_{k+1} = x_{k} + 
\begin{bmatrix}
v_{k}\cos\left(\psi_{k}\right)\Delta t \\
v_{k}\sin\left(\psi_{k}\right)\Delta t \\
0 \\
0 \\
0 \\
\end{bmatrix},
\end{align}
$$

which completes our definition of the deterministic part of the process model.

##### Uncertainty of the process model

The stochastic part of the process model is defined with respect to a process noise vector $\nu_{k} = \left[\nu_{a, k} \nu_{\ddot{\psi}, k}\right]^{\top}$, which describe the two independent scalar noise processes. The first noise process is the longitudinal acceleration noise $\nu_{a, k}$ which influences the longitudinal speed of the vehicle and it randomly changes its value at every time step $k$. The longitudinal acceleration is a normally-distributed white-noise with zero-mean and variance given by $\sigma_{a}^{2}$. The other noise process is the yaw acceleration noise $\nu_{\ddot{\psi}}$ which is also a normally-distributed white-noise with zero-mean and a variance given by $\sigma_{\ddot{\psi}}^{2}$, i.e., 
$$
\begin{align}
\nu_{a, k} &\sim \mathcal{N}\left(0, \sigma_{a}^{2}\right), \\
\nu_{\ddot{\psi}, k} &\sim \mathcal{N}\left(0, \sigma_{\ddot{\psi}}^{2}\right).
\end{align}
$$

The influence of the noise processes on the process model can be expressed in state-space form as:

$$
\begin{align}
x_{k+1} = x_{k} + 
\begin{bmatrix}
\frac{v_{k}}{\dot{\psi}_{k}} \left(\sin\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) - \sin\left(\psi_{k}\right)\right) \\
\frac{v_{k}}{\dot{\psi}_{k}} \left(-\cos\left(\psi_{k} + \dot{\psi}_{k}\Delta t\right) + \cos\left(\psi_{k}\right)\right) \\
0 \\ 
\dot{\psi}_{k} \Delta t\\
0
\end{bmatrix} +
\begin{bmatrix}
\frac{1}{2}\left(\Delta t\right)^{2} \cos\left(\psi_{k}\right)\cdot \nu_{a, k} \\
\frac{1}{2}\left(\Delta t\right)^{2} \sin\left(\psi_{k}\right)\cdot \nu_{a, k} \\
\Delta t \cdot \nu_{a, k} \\
\frac{1}{2}\left(\Delta t\right)^{2} \cdot \nu_{\ddot{\psi}, {k}} \\
\Delta t \cdot \nu_{\ddot{\psi}, k} \\
\end{bmatrix}
\end{align}
$$

and relies on the constant turn-rate assumption of the CTRV model, i.e., that the vehicle travels straight at time $t_{k+1}$ along the hypotenuse of the right-triangle formed in Figure 1 above. Note that this also assumes the effect of the yaw acceleration on the position to be zero, which in the non-zero case will result in the vehicle being on either side of the hypotenuse (since the non-zero yaw acceleration changes the radius of the circle formed by the trajectory the vehicle is travelling on).

With the CTRV model equations covered, we can now move onto the implementation of the Unscented Kalman filter (UKF). As we learned earlier, the UKF does not require us to compute the Jacobian matrix to linearise the non-linear process or measurement models. Instead, we form the representative sigma points sampled from a Gaussian distribution. These points will be then plugged into the non-linear process model equation above in order to estimate a new state mean and covariance using very similar equations to the standard Kalman filter for both the predict and innovation / update steps.

#### Alternatives

Before we close this discussion on the Constant Turn Rate and Velocity Magnitude (CTRV) model, we want to mention that there are several other common motion models one may consider. Those are:

* **Constant Turn Rate and Acceleration (CTRA)**;
* **Constant Steering Angle and Velocity (CSAV)**;
* **Constant Curvature and Acceleration (CCA)**.

Each model makes a different assumption about the vehicle's motion, and therefore choosing a model is quite an important step. For the remainder of this notebook, we make explicit reference to the CTRV model when designing our Unscented Kalman filter in C++.

## 2. Programming Task

TODO.

### 2.1. Unscented Kalman Filter in C++

#### 2.1.1. Generating Sigma Points

TODO.

In [9]:
### From J. Moran's `ukf.cc`

In [10]:
# TODO.

##### Testing the generate sigma points function

In [11]:
### From J. Moran's `5_tests.cc`

In [12]:
# TODO.

#### 2.1.2. UKF Augmentation

TODO.

In [17]:
### From J. Moran's `ukf.cc`

In [18]:
# TODO.

##### Testing the sigma point augmentation

In [19]:
### From J. Moran's `5_tests.cc`

In [ ]:
# TODO.

TODO.

#### 2.1.3. Sigma Point Prediction

TODO.

In [22]:
### From J. Moran's `ukf.cc`

In [ ]:
# TODO.

##### Testing the sigma point prediction

TODO.

#### 2.1.4. Prediction Step — Mean and Covariance

TODO.

In [27]:
### From J. Moran's `ukf.cc`

In [ ]:
# TODO.

###### Testing the mean and covariance prediction

In [29]:
### From J. Moran's `5_tests.cc`

In [ ]:
# TODO.

TODO.

## 3. Closing Remarks

##### Alternatives
* TODO.

##### Extensions of task
* TODO.

## 4. Future Work

- ⬜️ TODO.

## Credits

This assignment was prepared by Dominik Nuss, Andrei Vatavu et al., 2020 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).


References
* [] TODO. 

Helpful resources:
* TODO.